# Extract informations from seqs in fasta files

## Acquire seq length
### How to use:
First we need an `input directory`, containing fasta files and corressponding blast output, which are using suffix as 'basename'.fa and 'basename'.blast.out respectively.

In [ ]:
import pandas as pd
from Bio import SeqIO
import os
import glob

In [86]:
# input_fa="/lomi_home/gaoyang/software/CompRanking/benchmarking/MGE_benchmarking/plasclass/fq/2_assembly/DRR016448.contigs.fa"
# input_blastout="/lomi_home/gaoyang/software/CompRanking/benchmarking/MGE_benchmarking/plasclass/fq/2_assembly/DRR016448.contigs.fa.blast.out"
input_dir="/lomi_home/gaoyang/software/CompRanking/benchmarking/MGE_benchmarking/plasclass/fq/2_assembly/blastout_plasmid"
# b=filename=os.path.splitext(os.path.basename(input_fa))[0]
# b

In [13]:
def get_contigLen(input_fa):
    """
    input -> fasta file
    return -> {ID:length}
    """
    contigLen_dic={}
    #load fasta
    INPUT_FASTA=open(input_fa, "r")
    for record in SeqIO.parse(INPUT_FASTA, 'fasta'):
        contigLen_dic.setdefault(str(record.id), len(str(record.seq)))
    return contigLen_dic
# .split(" ")[0].strip(">")
def get_alignLen(input_blastout):
    """
    input -> blast .out file
    return -> {ID:alignment length}
    """
    aligenLen_dic={}
    INPUT_BLASTOUT=pd.read_csv(input_blastout, sep='\t', header=None)
    INPUT_BLASTOUT.columns = ['id', 'sub_id', 'identity', 'alignLen', 'mismat', 'gapOpens', 'qStart', 'qEnd', 'sStart', 'sEnd', 'eval', 'bit']
    for i, record in INPUT_BLASTOUT.iterrows():
        aligenLen_dic.setdefault(str(record['id']), [int(record['identity']), int(record['alignLen'])])
    return aligenLen_dic

def extract_subseq(seqName_list,input_fa):
    """
    input: seqID -> list; fasta file
    return: .length file and dic{>seqID \n "ATCG..."}
    """
    subseq_dic={}
    seqNameList=seqName_list
    INPUT_FASTA = open(input_fa, 'r')
    filename=os.path.splitext(os.path.basename(input_fa))[0]
    filename_path=os.path.splitext(input_fa)[0]
    with open(filename_path+".length", 'w') as f:
        for record in SeqIO.parse(INPUT_FASTA, 'fasta'):
            if str(record.id) in seqNameList:
                subseq_dic.setdefault(str(record.id),
                    str(record.seq))
                f.write(">"+str(record.id)+"_"+filename.spilt(".")[0]+" "+str(len(record.seq)) + '\n' + str(record.seq) + '\n')
            else:
                continue
        f.close()
    return subseq_dic

def extract_subseq_dic(seqName_list,input_fa):
    """
    input: seqID -> list; fasta file
    return: .length file and dic{>seqID \n "ATCG..."}
    """
    subseq_dic={}
    seqNameList=seqName_list
    INPUT_FASTA = open(input_fa, 'r')
    filename=os.path.splitext(os.path.basename(input_fa))[0]
    filename_path=os.path.splitext(input_fa)[0]
    # with open(filename_path+".length", 'w') as f:
    for record in SeqIO.parse(INPUT_FASTA, 'fasta'):
        if str(record.id) in seqNameList:
            subseq_dic.setdefault(str(record.id),
                len(str(record.seq)))
        else:
            continue
        # f.close()
    return subseq_dic

def extract_subseq_lengthFilter(seqName_list,input_fa,k):
    """
    input: fasta file, set k and list storing seqID
    extract exact seqs of length of interest from target file
    """
    subseq_dic={}
    seqNameList=seqName_list
    INPUT_FASTA = open(input_fa, 'r')
    filename=os.path.splitext(os.path.basename(input_fa))[0]
    filename_path=os.path.splitext(input_fa)[0]
    with open(filename_path+"_"+k+"k"+".fasta", 'w') as f:
        for record in SeqIO.parse(INPUT_FASTA, 'fasta'):
            if str(record.id) in seqNameList:
                f.write(">"+str(record.id)+"_"+filename.split(".")[0]+" "+ "len="+str(
                    len(record.seq)) + '\n' + str(record.seq) + '\n')
            else:
                continue
        f.close()
    return subseq_dic

def extract_subseq_nseq(number_of_seqs, input_fa):
    """
    input: fasta file
    extract exact number of seqs from target file
    """
    count=0
    ult=int(number_of_seqs)
    INPUT_FASTA = open(input_fa, 'r')
    filename_path=os.path.splitext(input_fa)[0]
    with open(filename_path+"_"+str(number_of_seqs)+"seqs.fasta", 'w') as f:
        for record in SeqIO.parse(INPUT_FASTA, 'fasta'):
            if count <= ult:
                f.write(">"+str(record.id) + '\n'  + str(
                    record.seq) + '\n')
                count += 1
            else:
                break
        f.close()

In [68]:
# contigLen_1=get_contigLen(input_fa)
# alignLen_1=get_alignLen(input_blastout)

# # print(get_contigLen(input_fa))
# # print(get_alignLen(input_blastout))
# tmp_list=[]
# for i in alignLen_1:
#     if alignLen_1[i][0] >= 95 and (alignLen_1[i][1] / contigLen_1[i]) >= 0.95:
#         tmp_list.append(str(i))

# a=tmp_list
# extract_subseq(a,input_fa)

{'k141_10': 'CTATCTCCACCTCAATAAAATACTCGTTACTAAAGGGGCCAGGGTTACCCGGGGAGGCGCTATTGCGTTATCCGGTAACAGCGGACGTTCATCCGGTCCTCATCTGCATTACGAGCTGGTCATCAATAACAATCCTGTTAACTCACTGGCGTTCCGGGCAGCGGCACCCGCTGATAACAAACTTGAACAGCATGCCTTTGCGCATGCCAGAGACTACGAACGATACCTGGACTGATAACGGGGCCGCGACGCGGCCCCGTCTGCCGGATTAATTTTTTTTATCGTTTTCACTTCCTTGATGTTGATGATCTCCATGCCCTCCGTGGCCGTGGAAAAGATGCATTAGC',
 'k141_60': 'AATATGGGGCTCATGTCCTGCTACGTAACCCGTCAGTAAAGCCCTGCTGCGCACCTGACGCTAAGCACTAACCCGCCTGCAGTTACCTGGTCGAATACAGCCCGCGAAGCTTTCTTGCCTGCGTCTGATGTGCTTCCGCACCGGCATTATTGACCTGCTCATGCACGAGAGCGGCTTTTTCTCCGGCATTCAGTTCGTTAAAAGAAGAAGACGAGGTCTTTGAATTTGCATCACTGCCGGACAGCATTTTTTTATGTTCCTCAATCATTTTCTGATGCGCATAGGACGCGCTGTTGTTCATAAATGAATGAGCAACAATGGCCCTTTCATGTTCATTCATTTCAGAGAATGAGGGCGTGT',
 'k141_66': 'ACCGTCCTGAAAAGCCAGACGGCAGACGGGCTGTATTACGCAGCCAGGTACTTGAACTGCATGGCATCAGCCACGGCTCTGCCGGAGCAAGAAGCATCGCCACAATGGCAACCCAGAGAGGCTACCAGATGGGGCGCTGGCTTGCTGGCAGACTCATGAAAGAGCTGGGGCTGGTCAGTTGCCAGCAGCCGACTCACCGGTATAAGCGTGGCGGTCATGAGCACGTTGCTATCCCGAATCATCTTGAG

In [85]:
#generate fitered sequence 
for i in glob.glob(input_dir+"/*fa"):
    file_basename=os.path.splitext(os.path.basename(i))[0]
    input_blastout=input_dir + "/" + file_basename + ".fa.blast.out"
    input_fa=i
    
    contigLen_1=get_contigLen(input_fa)
    alignLen_1=get_alignLen(input_blastout)

    # print(get_contigLen(input_fa))
    # print(get_alignLen(input_blastout))
    tmp_list=[]
    for i in alignLen_1:
        if alignLen_1[i][0] >= 95 and (alignLen_1[i][1] / contigLen_1[i]) >= 0.95:
            tmp_list.append(str(i))

    a=tmp_list
    extract_subseq(a,input_fa)
    
    
    
    
    # print(blast_outfile)

FileNotFoundError: [Errno 2] No such file or directory: '/lomi_home/gaoyang/software/CompRanking/benchmarking/MGE_benchmarking/plasclass/fq/2_assembly/DRR016448.contigs.fa.blast.out'

In [14]:
#length statistic for plasmid
input_dir="/lomi_home/gaoyang/software/CompRanking/benchmarking/MGE_benchmarking/plasclass/fq/2_assembly/blastout_plasmid"
length0_300=0
length300_500=0
length500_1000=0
length1000_5000=0
length5000_=0

for i in glob.glob(input_dir+"/*fa"):
    ls0_300=[]
    ls300_500=[]
    ls500_1000=[]
    ls1000_5000=[]
    ls5000_=[]
    file_basename=os.path.splitext(os.path.basename(i))[0]
    input_blastout=input_dir + "/" + file_basename + ".fa.blast.out"
    input_fa=i
    
    contigLen_1=get_contigLen(input_fa)
    alignLen_1=get_alignLen(input_blastout)
    tmp_list=[]
    for i in alignLen_1:
        if alignLen_1[i][0] >= 95 and (alignLen_1[i][1] / contigLen_1[i]) >= 0.95:
            tmp_list.append(str(i))

    a=tmp_list
    subseq_dic=extract_subseq_dic(a,input_fa)
    
    for j in subseq_dic:
        if subseq_dic[j] <=300:
            length0_300 += 1
            ls0_300.append(j)
        elif 300<=subseq_dic[j] < 500:
            length300_500 += 1
            ls300_500.append(j)
        elif 500 <= subseq_dic[j] < 1000:
            length500_1000 += 1
            ls500_1000.append(j)
        elif 1000 <= subseq_dic[j] < 5000:
            length1000_5000 += 1
            ls1000_5000.append(j)
        elif 5000 <= subseq_dic[j]:
            length5000_ += 1    
            ls5000_.append(j)
        
    # extract_subseq_lengthFilter(ls0_300,input_fa,"0.3")
    extract_subseq_lengthFilter(ls300_500,input_fa,"0.5")
    extract_subseq_lengthFilter(ls500_1000,input_fa,"1")
    extract_subseq_lengthFilter(ls1000_5000,input_fa,"5")
    extract_subseq_lengthFilter(ls5000_,input_fa,"lt5")

print(length0_300,
length300_500,
length500_1000,
length1000_5000,
length5000_)

575 637 470 944 522


In [15]:
#length statistic for chormosome
input_dir="/lomi_home/gaoyang/software/CompRanking/benchmarking/MGE_benchmarking/plasclass/fq/2_assembly/blastout_chromosome"

length0_300=0
length300_500=0
length500_1000=0
length1000_5000=0
length5000_=0

for i in glob.glob(input_dir+"/*fa"):
    ls0_300=[]
    ls300_500=[]
    ls500_1000=[]
    ls1000_5000=[]
    ls5000_=[]
    file_basename=os.path.splitext(os.path.basename(i))[0]
    input_blastout=input_dir + "/" + file_basename + ".fa.blast.out"
    input_fa=i
    
    contigLen_1=get_contigLen(input_fa)
    alignLen_1=get_alignLen(input_blastout)
    tmp_list=[]
    for i in alignLen_1:
        if alignLen_1[i][0] >= 95 and (alignLen_1[i][1] / contigLen_1[i]) >= 0.95:
            tmp_list.append(str(i))

    a=tmp_list
    subseq_dic=extract_subseq_dic(a,input_fa)
    
    for j in subseq_dic:
        if subseq_dic[j] <=300:
            length0_300 += 1
            ls0_300.append(j)
        elif 300<=subseq_dic[j] < 500:
            length300_500 += 1
            ls300_500.append(j)
        elif 500 <= subseq_dic[j] < 1000:
            length500_1000 += 1
            ls500_1000.append(j)
        elif 1000 <= subseq_dic[j] < 5000:
            length1000_5000 += 1
            ls1000_5000.append(j)
        elif 5000 <= subseq_dic[j]:
            length5000_ += 1    
            ls5000_.append(j)
        
    # extract_subseq_lengthFilter(ls0_300,input_fa,"0.3")
    extract_subseq_lengthFilter(ls300_500,input_fa,"0.5")
    extract_subseq_lengthFilter(ls500_1000,input_fa,"1")
    extract_subseq_lengthFilter(ls1000_5000,input_fa,"5")
    extract_subseq_lengthFilter(ls5000_,input_fa,"lt5")
        

print(length0_300,
length300_500,
length500_1000,
length1000_5000,
length5000_)

1404 3263 3181 7575 6531


extract target number of contigs

In [17]:
#generate fitered sequence 
input_dir="/lomi_home/gaoyang/software/CompRanking/benchmarking/MGE_benchmarking/plasclass/GT/chromosome"
for i in glob.glob(input_dir+"/*fa"):
    file_basename=os.path.splitext(os.path.basename(i))[0]
    input_fa=i
    extract_subseq_nseq(600, input_fa)
    
    

In [42]:
# aligenLen_dic={}
# INPUT_BLASTOUT=pd.read_csv(input_blastout, sep='\t', header=None)
# INPUT_BLASTOUT.columns = ['id', 'sub_id', 'identity', 'alignLen', 'mismat', 'gapOpens', 'qStart', 'qEnd', 'sStart', 'sEnd', 'eval', 'bit']
# INPUT_BLASTOUT
# for i, record in INPUT_BLASTOUT.iterrows():
#     aligenLen_dic.setdefault(str(record['id']), [int(record['identity']), int(record['alignLen'])])
# aligenLen_dic

In [ ]:
# DB_length=dict()
#     try:
#         for lines in open(file_name + '.length', 'r'):
#             DB_length.setdefault(str(lines.split('\t')[0]), len(str(lines.split('\t')[-1]).replace('\n','')))
#     except (IOError,FileNotFoundError):
#         Fasta_name = open(file_name, 'r')
#         f = open(file_name + '.length', 'w')
#         for record in SeqIO.parse(Fasta_name, 'fasta'):
#             f.write(str(record.id) + '\t'  + str(
#                 len(record.seq)) + '\n')
#             DB_length.setdefault(str(record.id), len(str(record.seq)))
#         f.close()
#     return DB_length